# How to visualize your VNET topology in Azure with PowerShell

> This notebook does work on Linux but does not work in Binder for some reason :(

## Scenario

Assume we have some network topology in Azure. The topology is dynamic, VNETs being added and removed dynamically, and at any particular moment, we can't say how it looks without exploring it. We want to visualize the topology somehow, at any point in time. Additionally, we want to highlight VNETs, where VNET peering was not configured correctly, as we know that this kind of thing happens from time to time due to some bugs in automated procedures we have. In addition to this, we also know that there are a few devices connected to some networks. Someone complained that one of these devices does not work as expected, and we want to see where it is

## Preparation

In this section we are going to prepare a lab, where we going to run a few experiments.

Before everything, we need to install a few modules, just in case. This step is also useful in Binder, where there are no such modules

In [ ]:
Install-Module -Name PSQuickGraph -AllowPrerelease -RequiredVersion "2.0.2-alpha"
Install-Module -Name ipmgmt
Install-Module -Name Az -Scope CurrentUser -Repository PSGallery

And load them

In [ ]:
Import-Module ipmgmt
Import-Module PSQuickGraph -RequiredVersion "2.0.2"

Log in to Azure. It will fail in Binder ...

In [ ]:
Login-AzAccount

and it will ask you to use this

In [ ]:
Connect-AzAccount -UseDeviceAuthentication

set the initial constants

In [ ]:
$rgName = "vnet-test"
$region = "eastus2"
$baseNet = "10.96.0.0/16"

Create the resource group for our experiment, just in case.

In [ ]:
New-AzResourceGroup -Name $rgName -Location $region

Now, we can prepare a set of parameters for VNETs. We are using the `ipmgmt`, which we looked at in the [previous](https://github.com/eosfor/scripting-notes/blob/main/notebooks/ru/ipmgmt.ipynb) notebook. Here we simply make an array of hashtables and pass it to the `Get-VLSMBreakdown` cmdlet, which returns us the IP ranges we are looking for. We just iterate from "A" to "K" and for each character create an element in this array.

In [ ]:
$vnets = "A".."K" | % { @{type = "VNET-$_"; size = (256-2)} }

Now we just pass the array to the commandlet, and it returns a list of ranges we need.

In [ ]:
Get-VLSMBreakdown -Network $baseNet -SubnetSize $vnets | ? type -ne 'reserved' | 
    ft type, network, netmask, *usable, cidr -AutoSize


type   Network    Netmask       FirstUsable LastUsable   Usable Cidr
----   -------    -------       ----------- ----------   ------ ----
VNET-K 10.96.10.0 255.255.255.0 10.96.10.1  10.96.10.254    254   24
VNET-J 10.96.9.0  255.255.255.0 10.96.9.1   10.96.9.254     254   24
VNET-I 10.96.8.0  255.255.255.0 10.96.8.1   10.96.8.254     254   24
VNET-H 10.96.7.0  255.255.255.0 10.96.7.1   10.96.7.254     254   24
VNET-G 10.96.6.0  255.255.255.0 10.96.6.1   10.96.6.254     254   24
VNET-F 10.96.5.0  255.255.255.0 10.96.5.1   10.96.5.254     254   24
VNET-E 10.96.4.0  255.255.255.0 10.96.4.1   10.96.4.254     254   24
VNET-D 10.96.3.0  255.255.255.0 10.96.3.1   10.96.3.254     254   24
VNET-C 10.96.2.0  255.255.255.0 10.96.2.1   10.96.2.254     254   24
VNET-B 10.96.1.0  255.255.255.0 10.96.1.1   10.96.1.254     254   24
VNET-A 10.96.0.0  255.255.255.0 10.96.0.1   10.96.0.254     254   24



At this point, we can create these networks. To simplify, we create only one subnet in each, but nothing stops us from using `Get-VLSMBreakdown` to break each VNET into subnets if we need to.

In [ ]:
Get-VLSMBreakdown -Network $baseNet -SubnetSize $vnets | ? type -ne 'reserved' | % {
    $addressPrefix = "$($_.network)/$($_.cidr)"
    $subnet = New-AzVirtualNetworkSubnetConfig -Name "default" -AddressPrefix $addressPrefix
    New-AzVirtualNetwork -Name $_.type -ResourceGroupName $rgName -Location $region -AddressPrefix $addressPrefix -Subnet $subnet | out-null
}

Adding VNET peering

In [ ]:
$nets = Get-AzVirtualNetwork -ResourceGroupName $rgName

We are going to use hub-and-spoke topology, as we usually do in Landing Zones. Ans to simulate a misconfiguration, we skip one VNET peering link

In [ ]:
$hub = $nets[0]
$spokes = $nets[1..($nets.count-1)]

$spokes | % {
    Add-AzVirtualNetworkPeering `
        -Name "$($hub.Name)-$($_.Name)" `
        -VirtualNetwork $hub `
        -RemoteVirtualNetworkId $_.Id | out-null
}

$spokes | select -Skip 1 |  % {      
    Add-AzVirtualNetworkPeering `
        -Name "$($_.Name)-$($hub.Name)" `
        -VirtualNetwork $_ `
        -RemoteVirtualNetworkId $hub.Id | out-null
}

Add-AzVirtualNetworkPeering: 
Line |
   5 |      Add-AzVirtualNetworkPeering `
     |      ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Peering with the specified name already exists
Add-AzVirtualNetworkPeering: 
Line |
   5 |      Add-AzVirtualNetworkPeering `
     |      ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Peering with the specified name already exists
Add-AzVirtualNetworkPeering: 
Line |
   5 |      Add-AzVirtualNetworkPeering `
     |      ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Peering with the specified name already exists
Add-AzVirtualNetworkPeering: 
Line |
   5 |      Add-AzVirtualNetworkPeering `
     |      ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Peering with the specified name already exists
Add-AzVirtualNetworkPeering: 
Line |
   5 |      Add-AzVirtualNetworkPeering `
     |      ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Peering with the specified name already exists
Add-AzVirtualNetworkPeering: 
Line |
   5 |      Add-AzVirtualNetworkPeering `
     |      ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    

Error: System.OperationCanceledException: Command :SubmitCode: $hub = $nets[0]
$spokes = $nets[1..($nets.count-1) ... cancelled.

To model the network devices we add a few NICs into random VNETs. This is a lab, and we don't need full VMs for that. Well, in this case for the sake of the experiment we need a NIC to exist in a specific VNET. For that we use `begin`, `process`, and `end` script block parameters of the `Foreach-Object`. In this case, we initialize the variable in the `begin` script block and use it in the `process`, so it is guaranteed to be initialized with the correct value.

In [ ]:
1..4 | % {$idx = 0} {
    $vnetForNIC = $spokes[$idx]
    New-AzNetworkInterface -Name "NetworkInterface-$idx" -ResourceGroupName $rgName -Location $region -SubnetId $vnetForNIC.Subnets[0].Id
    $idx = Get-Random -Minimum 1 -Maximum ($spokes.Count-1)
}

## Experiment

If all the above worked well, we are ready for the experiment. Let us read everything again, and create an empty graph

In [ ]:
$vnets = Get-AzVirtualNetwork -ResourceGroupName $rgName
$g = New-Graph

We need to fill the graph with data, and we are going to do it in a few iterations. At fist, we will add VNETs, then VNET peerings, and at the end - NICs

#### Adding VNETs

We have everything we need to add VNETs to the graph. However, there is a little problem. We can add any objects to the graph, but when we do it, the graph "converts" it into a special type - `PSGraph.Model.PSVertex`. There is the `Label` property in this type, and it is a key property for comparison operations, search and visualization. Another property of interest is the `OriginalObject` property. It has a link to the origunal object, which was used to create the vertex. To initialize the `Label` property the `ToString()` method is being called on each object. In cas of `PSVirtualNetwork` this call returns the name of the type, but we need unique names as labels. To overcome this we have a constructor for the `PSGraph.Model.PSVertex` whch take as `Label` as `string` and the original object. This way we can override the labeling mechanism and provide our own labels.

In [ ]:
$vnets | % {
    Add-Vertex -Graph $g -Vertex ([PSGraph.Model.PSVertex]::new($_.Id, $_))
}

#### Adding VNET peerings

At this point, we have VNETs in the graph. Now we need to add connections between these networks. To do that we iterate through the graph vertices, pull the `OriginalObject` property of each of them, and peek into it to find VNET's peerings. Each peering stores the `resourceID` of the remote VNET. As far as we use VNET resource IDs as labels on the graph, we can just search the graph for the corresponding vertex, and use it as a target for the graph edge. So the source is the current VNET, and the target is the VNET pointed by the peering information.

In [ ]:
foreach ($v in $g.Vertices){
    foreach($p in $v.OriginalObject.VirtualNetworkPeerings) {
        foreach ($rvn in $p.RemoteVirtualNetwork) {
            $targetVertex = $g.Vertices.Where({$_.Label -eq $rvn.id})[0]
            Add-Edge -From $v -To $targetVertex -Graph $g
        }
    }
}

Now we have networks and links between them. So we can visualize the topology. But we want to highlight misconfigured networks. Here we cheat a bit because we know how exactly the network was misconfigured. In the real scenario, you would test for more than one misconfigured thing. But in our case, just for simplicity, we are running through vertices again, and colorizing those which does not have outgoing edges.

In [ ]:
$g.Vertices | % { if ( $g.OutDegree($_) -eq 0 ) 
                    { $_.GVertexParameters.Fillcolor = [QuikGraph.Graphviz.Dot.GraphvizColor]::OrangeRed  } 
                }

#### Adding NICs

The last step we need to to is to add NICs to our graph. We read them first

In [ ]:
$nics = Get-AzNetworkInterface -ResourceGroupName $rgName

Each NIC has a reference to its subnet in the `IpConfigurations` property, stored as a `resourceID`. It is a string, so we can just cut everything after `/subnets/` and use the rest of the string as a reference to a VNET.

In [ ]:
$nics | % {
    $vnetID = $_.IpConfigurations[0].Subnet.Id -replace "/subnets/.+", ""
    $targetVertex = $g.Vertices.Where({$_.Label -eq $vnetID})[0]
    Add-Edge -Graph $g -From ([PSGraph.Model.PSVertex]::new($_.name, $_)) -To $targetVertex
}

Now, we have everything we need in the graph. As the last step we want to color **NICs only**, so they are easily visible on the visualization

In [ ]:
$g.Vertices | 
    ? {$_.OriginalObject.GetType() -eq [Microsoft.Azure.Commands.Network.Models.PSNetworkInterface]} | 
    % {  $_.GVertexParameters.Fillcolor = [QuikGraph.Graphviz.Dot.GraphvizColor]::WhiteSmoke }

#### Exporting and visualizing

Now we just export the data into SVG and [Graphviz format](https://www.graphviz.org/doc/info/lang.html).

In [ ]:
Export-Graph -Graph $g -Path "$($env:TEMP)\topology.svg" -Format MSAGL_MDS
Export-Graph -Graph $g -Path "$($env:TEMP)\topology.gv" -Format Graphviz

In [ ]:
using System.IO;

var path = Path.GetTempPath();
var svg = File.ReadAllText($"{path}\\topology.svg");
var gv = File.ReadAllText($"{path}\\topology.gv");
svg.DisplayAs("text/html"); // this will embed a big SVG into the notebook, and it is not very convenient

<?xml version="1.0" encoding="utf-8"?>
<!--SvgWriter version 0.0.0.0-->
 
 
 <!--Edges-->
 
 
 <path fill="none" stroke="#000000" stroke-opacity="1" stroke-width="1" d="M 1104.94801700104 -230.75543380545 L 1079.21397550769 -222.72967873329 A 73.16941149983,73.16941149983 180 0 1 1071.28760436744 -220.73583600751 A 94.5526642997,94.5526642997 180 0 1 1063.19774232848 -219.53558547203 L 271.33044829785 -136.86005383246 A 27.24095102952,27.24095102952 0 0 0 257.07162536719 -130.98157045492 A 16.33020215358,16.33020215358 0 0 0 250.98900486541 -117.89958990189 L 252.29819082276 -59.18813810661 A 8.54064051219,8.54064051219 180 0 1 249.30174618173 -52.49943603959 A 10.94069336277,10.94069336277 180 0 1 242.15783081682 -49.88326227631 L 79.29406604071 -50.54470291157 A 1.82119278366,1.82119278366 180 0 1 79.16675860422 -50.54967638133 L 79.16678800399 -50.55009110973 L 228.95897706934 -40.42231362115" />
 
 <path fill="none" stroke="#000000" stroke-opacity="1" stroke-width="1" d="M -747.12259463251 209.31577467765 L -686.87906887635 194.47008259238 A 224.38550893732,224.38550893732 180 0 1 -677.561801405 192.38327072338 A 730.62907361022,730.62907361022 180 0 1 -668.18818225705 190.55600160461 L -343.85380559057 129.52858121723 A 18.98188557224,18.98188557224 0 0 0 -337.56893314039 127.13354437212 A 15.96823758646,15.96823758646 0 0 0 -332.69303639157 122.56380915108 L -327.91716331221 115.6866753508 A 19.05322467552,19.05322467552 0 0 0 -324.57621875947 103.27513096272 A 49.19150319895,49.19150319895 0 0 0 -327.5143082768 90.0742763726 L -350.12741530712 29.42718847778 A 7.96083354816,7.96083354816 180 0 1 -346.09969614957 19.46262018247 A 33.66891321912,33.66891321912 180 0 1 -332.24472505611 16.18063958397 L -102.8376112863 11.69777708169 A 10.30863951949,10.30863951949 180 0 1 -100.32691758474 11.95779641864 A 4.43433717169,4.43433717169 0 0 0 -97.88567695076 11.82745330461 L -73.83000056211 3.51742904573" />
 
 
 
 <path fill="none" stroke="#000000" stroke-opacity="1" stroke-width="1" d="M -237.21669726874 -210.54301354233 L -201.09947312765 -197.13757024612 A 81.8193275834,81.8193275834 0 0 0 -192.42261247415 -194.45469844871 A 102.95756796828,102.95756796828 0 0 0 -183.51678952403 -192.64755719656 L 224.90799081741 -128.38922999225 A 18.2896287045,18.2896287045 180 0 1 232.65173888769 -125.23627799072 A 8.82027167495,8.82027167495 180 0 1 236.35680434447 -118.46135368254 L 239.17948529125 -58.91501940213 A .91846456055,.91846456055 180 0 1 238.85383217365 -58.16912566067 A 1.22445466329,1.22445466329 180 0 1 238.05277862392 -57.88114430987 L 79.67406431396 -59.44864279433 A .38279058373,.38279058373 180 0 1 79.6088548659 -59.45489380264 A .17170578407,.17170578407 0 0 0 79.54428031558 -59.45438168646 L -9.33374415094 -41.70600451985" />
 
 
 
 
 
 
 
 <path fill="none" stroke="#000000" stroke-opacity="1" stroke-width="1" d="M -43.54526974003 .25226811224 L -96.64898712312 15.20297173918 A 7.35616426691,7.35616426691 180 0 1 -100.08242078373 15.33594989246 A 16.39255712212,16.39255712212 0 0 0 -103.58169519287 15.02142502709 L -332.20269210067 19.07486562696 A 23.81569668272,23.81569668272 0 0 0 -342.46443275311 21.60205679276 A 6.16243164733,6.16243164733 0 0 0 -345.51561846046 29.14752084277 L -324.47956255648 89.17894966298 A 193.0554638811,193.0554638811 180 0 1 -319.88646338459 104.2245183395 A 16.92625372733,16.92625372733 180 0 1 -322.43920245119 118.27014869916 L -327.9467161216 126.06511081626 A 22.51244148268,22.51244148268 180 0 1 -334.42988639412 132.1827347754 A 24.22203529686,24.22203529686 180 0 1 -342.7577835423 135.42762653573 L -667.1657418917 196.46824446999 A 714.3197569075,714.3197569075 0 0 0 -676.33214911116 198.25511564248 A 219.4302276689,219.4302276689 0 0 0 -685.44344869594 200.29580097706 L -712.33679347686 206.92307437697" />
 
 <path fill="none" stroke="#000000" stroke-opacity="1" stroke-width="1" d="M -85.21089853703 .25226811224 L -99.1223667784 8.45193487005 A 1.74172915

So we are going to do it a bit differently. We will use HTML and [vis.js](https://visjs.org/). First we prepare a canvas

In [ ]:
#!html
<div id="mynetwork" style="height: 800px;"></div>

And then visualize the graph. pay attention to the line `#!share --from csharp gv`. It is a convenient way to "share" variables within the same kernel between different languages. In this case, we used C# to read the Graphviz file into the `gv` variable, and then use that variable within the block of JavaScript!

In [ ]:
#!js
#!share --from csharp gv

visRequire = interactive.configureRequire({
    paths: {
        visjs: "https://visjs.github.io/vis-network/standalone/umd/vis-network.min"
    }
});
    
visRequire(["visjs"], visjs => {
    
    var container = document.getElementById("mynetwork");
    var dot = gv;
    var parsedData = visjs.parseDOTNetwork(dot);

    var data = {
        nodes: parsedData.nodes,
        edges: parsedData.edges
    };
    var options = parsedData.options;
    options = {
        physics: {
            solver: "forceAtlas2Based",
            enabled: false,
            forceAtlas2Based: {
                theta: 0.5,
                gravitationalConstant: -50,
                centralGravity: 0.01,
                springConstant: 0.08,
                springLength: 100,
                damping: 0.4,
                avoidOverlap: 0.7
              },
            barnesHut: {
                theta: 0.5,
                gravitationalConstant: -2000,
                centralGravity: 0.3,
                springLength: 95,
                springConstant: 0.04,
                damping: 0.09,
                avoidOverlap: 0
            }
        },
        interaction: { hover: true, zoomView: true },
        layout: { randomSeed: 'Mickey' }
    }

    var network = new visjs.Network(container, data, options); 
    network.stabilize(600)
});

Voila! Here we see, that the misconfigured VNET is marked red. And it has a NIC attached to it. Which is, potentially, the root cause of the hypothetical problem :)